# Julia QR
Julia (along with any other high performance library) avoids extraneous computations. The underlying return format for efficient QR code is probably not what you expect. The structure access "." tools compute the pieces you need.  Full documentation is available. 

In [1]:
using LinearAlgebra
(m,n)=(12,5)
A=rand(m,n)
@time QROut=qr(A)
display(QROut.Q)
display(Matrix(QROut))

  0.065220 seconds (77.46 k allocations: 5.018 MiB, 99.79% compilation time)


12×12 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}, Matrix{Float64}}

12×5 Matrix{Float64}:
 0.991736   0.444736   0.838895   0.283683   0.808687
 0.0971169  0.402128   0.294043   0.960827   0.548582
 0.156443   0.0604203  0.632355   0.250303   0.630161
 0.172287   0.30577    0.0581267  0.305484   0.419571
 0.874268   0.963673   0.961341   0.0945897  0.107062
 0.903079   0.936769   0.255614   0.0701265  0.504467
 0.646611   0.14039    0.280426   0.682581   0.982301
 0.1134     0.207517   0.410462   0.509712   0.975198
 0.9255     0.941063   0.579335   0.672515   0.441378
 0.121809   0.695961   0.3097     0.714781   0.723043
 0.472171   0.384835   0.651229   0.0646235  0.687732
 0.343396   0.243096   0.2441     0.856875   0.30162

It is worth noting that the Q factor acts like a square matrix!  It is much faster to use the implicit Q factor than it is to build the Q and then multiply. 

In [7]:
using LinearAlgebra
(m,n)=(1234,25)
A=rand(m,n); vm=rand(m); vn=vm[1:n];
@time QROut=qr(A)
@time Q=QROut.Q
@time QMat=Matrix(Q);

  0.000727 seconds (5 allocations: 251.266 KiB)
  0.000015 seconds (1 allocation: 32 bytes)
  0.000800 seconds (3 allocations: 246.172 KiB)


In [8]:
@time uFast=Q*vn
@time uSlow=QMat*vn
display(@time norm(uFast-uSlow))
@time Q*vm;

  0.000156 seconds (4 allocations: 10.156 KiB)
  0.000304 seconds (1 allocation: 9.812 KiB)
  0.000016 seconds (2 allocations: 9.828 KiB)


7.367591498653301e-16

  0.000105 seconds (2 allocations: 10.062 KiB)


The "extra" columns of "Q" span the orthogonal complement of the columns space of A.  This could be useful if you want an efficient way to efficiently expand the column space of A.     

In [11]:
using SparseArrays
m=1234
@time em = spzeros(m)
@time em[m]=1.0
@time uPerp=Q*em
@time norm(A'*uPerp)

  0.000007 seconds (3 allocations: 160 bytes)
  0.000011 seconds (2 allocations: 160 bytes)
  0.000162 seconds (3 allocations: 19.875 KiB)
  0.000318 seconds (3 allocations: 288 bytes)


1.4106118103708833e-15

Sparse arrays are stored quite differently from dense arrays! Julia gives you a bad cartoon of a sparse matrix.

In [15]:
using SparseArrays
(m,n)=(1234,2342)
v=rand(n)
@time A=zeros(m,n); A[m,n]=2.3; A[1,1]=3.4
@time As=sparse([m,1],[n,1],[2.3,3.4])
@time u=A*v
@time w=As*v;
norm(w-u)

  0.005802 seconds (2 allocations: 22.049 MiB)
  0.000062 seconds (18 allocations: 47.328 KiB)
  0.002175 seconds (1 allocation: 9.812 KiB)
  0.000014 seconds (1 allocation: 9.812 KiB)


0.0